In [56]:
import pandas as pd
import numpy as np
import json

In [186]:
D = pd.read_csv('data/4clusters.csv', header=None, skiprows=1)
D

,0,1
0,41,45
1,39,44
2,42,43
3,44,43
4,10,42
5,38,42
6,8,41
7,41,41
8,13,40
9,45,40


In [19]:
D.stack().idxmin()

(17, 0)

In [226]:
def link_dist(C_ij, C_ik, link = "average") -> float:
    dists = []
    D_ij = pd.concat([pd.Series(leaf["data"].split(',')).astype(float) for leaf in flatten(C_ij)], axis=1).T
    D_ik = pd.concat([pd.Series(leaf["data"].split(',')).astype(float) for leaf in flatten(C_ik)], axis=1).T
    for j in D_ij.index:
        for k in D_ik.index:
            dists.append(np.sqrt(((D_ij.loc[j] - D_ik.loc[k])**2).sum()))
    dists = pd.Series(dists)
    if link == "average":
        return dists.mean()
    elif link == "single":
        return dists.min()
    elif link == "complete":
        return dists.max()
    elif link == "centroid":
        return np.sqrt(((D_ij.mean() - D_ik.mean())**2).sum())

In [105]:
def flatten(node):
    flat = []
    if node["type"] == "leaf":
        flat.append(node)
    elif node["type"] in ("node", "root"):
        subnodes = node["nodes"]
        for n in subnodes:
            flat.extend(flatten(n))
    return flat

In [139]:
def get_heights(node):
    heights = []
    if node["type"] == "leaf":
        return heights
    elif node["type"] in ("node", "root"):
        heights.append(node["height"])
        subnodes = node["nodes"]
        for n in subnodes:
            heights.extend(get_heights(n))
        heights.sort()
        heights.reverse()
        return heights

In [140]:
get_heights(tree)

[30.084978393339334,
 26.530576664143805,
 13.331252931982712,
 12.433279908584838,
 11.72626389846176,
 10.395753610573419,
 6.9184175556797625,
 6.153440464842388,
 6.0,
 5.933986244776953,
 5.493508991637533,
 5.385164807134504,
 5.105625978844702,
 5.052916037910906,
 5.0,
 4.57101383701828,
 4.165973108952852,
 4.106558690472924,
 3.81720680758398,
 3.64269164289302,
 3.5615528128088303,
 3.525729106681236,
 3.1622776601683795,
 3.1622776601683795,
 3.0,
 2.8284271247461903,
 2.699172818834085,
 2.699172818834085,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.0,
 2.0,
 1.4142135623730951,
 0.0]

In [228]:
def make_tree(D):
    C = {}

    i = 1
    C[i] = [{"type": "leaf", "height": 0, "data": ",".join(D.loc[c].astype(str))} for c in list(D.index)]
    while len(C[i]) > 1:
        n_range = range(len(C[i]))
        d = pd.DataFrame(index=n_range, columns=n_range).astype(float)
        for j in n_range:
            for k in range(j+1, len(C[i])):
                d.iloc[j, k] = link_dist(C[i][j], C[i][k])
        d_stack = d.stack()
        s, r = d_stack.idxmin()
        height = d_stack.min()

        C[i+1] = []
        for j in n_range:
            if j != r and j != s:
                C[i+1].append(C[i][j])
            elif j == r:
                C[i+1].append({"type": "node", "height": height, "nodes": [C[i][r], C[i][s]]})
        i += 1

    tree = C[i][0]
    tree["type"] = "root"
    return tree

In [227]:
C = {}
heights = []

i = 1
C[i] = [{"type": "leaf", "height": 0, "data": ",".join(D.loc[c].astype(str))} for c in list(D.index)]
while len(C[i]) > 1:
    n_range = range(len(C[i]))
    d = pd.DataFrame(index=n_range, columns=n_range).astype(float)
    for j in n_range:
        for k in range(j+1, len(C[i])):
            d.iloc[j, k] = link_dist(C[i][j], C[i][k])
    d_stack = d.stack()
    s, r = d_stack.idxmin()
    height = d_stack.min()

    C[i+1] = []
    for j in n_range:
        if j != r and j != s:
            C[i+1].append(C[i][j])
        elif j == r:
            C[i+1].append({"type": "node", "height": height, "nodes": [C[i][r], C[i][s]]})
    i += 1

KeyboardInterrupt: 

In [99]:
tree = C[i][0]
tree["type"] = "root"
tree

{'type': 'root',
 'height': 30.084978393339334,
 'nodes': [{'type': 'node',
   'height': 26.530576664143805,
   'nodes': [{'type': 'node',
     'height': 13.331252931982712,
     'nodes': [{'type': 'node',
       'height': 5.493508991637533,
       'nodes': [{'type': 'node',
         'height': 2.0,
         'nodes': [{'type': 'leaf', 'height': 0, 'data': '42,9'},
          {'type': 'leaf', 'height': 0, 'data': '40,9'}]},
        {'type': 'node',
         'height': 3.81720680758398,
         'nodes': [{'type': 'node',
           'height': 3.1622776601683795,
           'nodes': [{'type': 'leaf', 'height': 0, 'data': '37,10'},
            {'type': 'leaf', 'height': 0, 'data': '38,13'}]},
          {'type': 'leaf', 'height': 0, 'data': '34,11'}]}]},
      {'type': 'node',
       'height': 11.72626389846176,
       'nodes': [{'type': 'node',
         'height': 10.395753610573419,
         'nodes': [{'type': 'node',
           'height': 5.933986244776953,
           'nodes': [{'type': 'node

In [103]:
heights.reverse()
heights

[30.084978393339334,
 26.530576664143805,
 13.331252931982712,
 12.433279908584838,
 11.72626389846176,
 10.395753610573419,
 6.9184175556797625,
 6.153440464842388,
 6.0,
 5.933986244776953,
 5.493508991637533,
 5.385164807134504,
 5.105625978844702,
 5.052916037910906,
 5.0,
 4.57101383701828,
 4.165973108952852,
 4.106558690472924,
 3.81720680758398,
 3.64269164289302,
 3.5615528128088303,
 3.525729106681236,
 3.1622776601683795,
 3.1622776601683795,
 3.0,
 2.8284271247461903,
 2.699172818834085,
 2.699172818834085,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 2.0,
 2.0,
 1.4142135623730951,
 0.0]

In [50]:
D.loc[[2]]

,0,1
2,42,43


In [104]:
[1, [2, 3], [4, [5, 6]], [7, 8], 9]

[1, [2, 3], [4, [5, 6]], [7, 8], 9]

In [146]:
def cut_tree(tree, threshold, by = "height"):
    queue = [tree]

    condition = 'False'
    if by == "height":
        condition = 'queue[0]["height"] > threshold'
    elif by == "nclust":
        condition = 'len(queue) < threshold'

    while eval(condition):
        poptree = queue.pop(0)
        queue.extend(poptree["nodes"])
        queue = sorted(queue, key=lambda n: -n["height"])
    return queue

In [220]:
def dist(x, y):
    return np.sqrt(((x - y)**2).sum())

In [246]:
def hclust(D, threshold, by = "height", std = True, restrict = None, row_id = None):
    D_orig = D.copy()
    cols = list(D_orig.columns)
    if restrict is not None:
        cols_orig = D_orig.columns
        cols = [cols_orig[i] for i in range(len(cols_orig)) if restrict[i] == 1]
        D = D.loc[:, cols]
    D_mean, D_std = 0, 1
    if std:
        D_mean, D_std = D.mean(), D.std()
        D = (D - D_mean) / D_std
    D = D.sort_values(cols)
    D_orig = D_orig.reindex(list(D.index))

    tree = make_tree(D)
    cut = cut_tree(tree, threshold, by=by)

    clusts = {i: [leaf["data"].split(",") for leaf in flatten(cut[i])] for i in range(len(cut))}
    clusts = {i: pd.DataFrame(clusts[i]).map(pd.to_numeric) for i in clusts.keys()}
    for i in clusts.keys():
        clusts[i]['cluster'] = i
    D = pd.concat(clusts.values()).sort_values(cols)
    D.index = D_orig.index
    D.columns = cols + ['cluster']

    D_clust = D_orig.copy()
    D_clust['cluster'] = D['cluster']
    if row_id is not None:
        D_clust = D_clust.set_index(row_id)
        D_clust.index.name = None

    centroids = D.groupby('cluster').mean()
    centroids_unstd = centroids * D_std + D_mean

    err = D.apply(lambda x: dist(x.drop('cluster'), centroids.loc[x.loc['cluster']]), axis=1)
    err = pd.DataFrame({'err': err, 'cluster': D['cluster']})
    err['err_sq'] = err['err']**2
    err = err.groupby('cluster')
    err_stats = pd.DataFrame({'min': err['err'].min(),
                              'max': err['err'].max(),
                              'avg': err['err'].mean(),
                              'sse': err['err_sq'].sum()})

    interclust = pd.DataFrame(index=list(centroids.index), columns=list(centroids.index))
    for i in interclust.index:
        for j in interclust.columns:
            interclust.loc[i, j] = dist(centroids.loc[i], centroids.loc[j])

    return {'clustered original data': D_clust.sort_index(),
            'clustered processed data': D.sort_index(),
            'tree': tree,
            'threshold': threshold,
            'cut type': by,
            'standardized': std,
            'centroids': centroids,
            'unstandardized centroids': centroids_unstd,
            'distance summary': err_stats,
            'intercluster distances': interclust}


In [247]:
hclust(D, 5, by="nclust")

{'clustered original data':      0   1  cluster
 0   41  45        4
 1   39  44        4
 2   42  43        4
 3   44  43        4
 4   10  42        1
 5   38  42        4
 6    8  41        1
 7   41  41        4
 8   13  40        1
 9   45  40        4
 10   7  39        1
 11  38  39        4
 12  42  39        4
 13   9  38        1
 14  12  38        1
 15  19  38        2
 16  25  38        2
 17   6  37        1
 18  13  35        1
 19   9  34        1
 20  12  34        1
 21  32  27        0
 22  26  25        0
 23  39  24        0
 24  34  23        0
 25  37  23        0
 26  22  22        0
 27  38  21        0
 28  35  20        0
 29  31  18        0
 30  26  16        0
 31  31  13        0
 32  26  16        0
 33  38  13        3
 34  29  11        0
 35  34  11        3
 36  37  10        3
 37  40   9        3
 38  42   9        3,
 'clustered processed data':            0         1  cluster
 0   0.994719  1.296488        4
 1   0.836052  1.214137        4
 2   

In [183]:
hclust(D, 5, by="nclust")

,0,1,cluster,0,1,0,1
17,-1.781951,0.637686,1,-1.781951,0.637686,-1.781951,0.637686
10,-1.702618,0.802386,1,-1.702618,0.802386,-1.702618,0.802386
6,-1.623284,0.967087,1,-1.623284,0.967087,-1.623284,0.967087
19,-1.543951,0.390636,1,-1.543951,0.390636,-1.543951,0.390636
13,-1.543951,0.720036,1,-1.543951,0.720036,-1.543951,0.720036
4,-1.464617,1.049437,1,-1.464617,1.049437,-1.464617,1.049437
20,-1.305951,0.390636,1,-1.305951,0.390636,-1.305951,0.390636
14,-1.305951,0.720036,1,-1.305951,0.720036,-1.305951,0.720036
18,-1.226617,0.472986,1,-1.226617,0.472986,-1.226617,0.472986
8,-1.226617,0.884737,1,-1.226617,0.884737,-1.226617,0.884737


In [152]:
[1, 2, 3, 4][1, 2]

<>:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
C:\Users\hsu_m\AppData\Local\Temp\ipykernel_14448\165435532.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  [1, 2, 3, 4][[1, 2]]
C:\Users\hsu_m\AppData\Local\Temp\ipykernel_14448\165435532.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  [1, 2, 3, 4][[1, 2]]
C:\Users\hsu_m\AppData\Local\Temp\ipykernel_14448\165435532.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  [1, 2, 3, 4][[1, 2]]


TypeError: list indices must be integers or slices, not list